In [1]:
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java 
!java -version
from __future__ import print_function

import sys
if sys.version >= '3':
    long = int

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
import csv
#import org.apache.spark.sql.Column
import pyspark.sql.functions as f

     |████████████████████████████████| 217.8MB 59kB/s 
     |████████████████████████████████| 204kB 47.5MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=e3cbde7fd45618bf204c5f0766d61fdb75da89cf4d20d76dc81243dadbaa6052
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)


In [0]:
sc = SparkContext("local", "RecommendationSystem")
spark = SparkSession(sc)

In [0]:
lines = spark.read.text("./new_user_rating.csv").rdd

In [0]:
#Get rid of the hearder
header = lines.first()
header = sc.parallelize([header]) 
lines = lines.subtract(header)

In [0]:
parts = lines.map(lambda row: row.value.split(","))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[1]), jobId=int(p[2]),rating=float(p[3])))
ratings = spark.createDataFrame(ratingsRDD)

In [24]:
#Split to training and test to a ritio of 8:2
(training, test) = ratings.randomSplit([0.8, 0.2])
tpanda = training.toPandas()
testpand = test.toPandas()
# Build the recommendation model using ALS on the training data
# Set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# Change implict to true
als = ALS(maxIter=5, regParam=0.01, implicitPrefs=True, userCol="userId", itemCol="jobId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 job recommendations for each user
userRecs = model.recommendForAllUsers(5)

Root-mean-square error = 1.0420623624206533


In [19]:
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   463|[[199125, 4.15083...|
|   496|[[1104251, 5.0441...|
|  1645|[[222842, 10.4480...|
|  5156|[[519704, 18.1769...|
| 15846|[[842341, 4.99780...|
| 16339|[[372741, 4.16197...|
| 18911|[[186978, 5.39721...|
| 18944|[[205937, 3.47751...|
| 21220|[[847325, 3.54416...|
| 53191|[[657615, 2.85625...|
| 61051|[[911258, 4.04819...|
| 78064|[[1030281, 5.5502...|
|120861|[[1030281, 2.6514...|
|130062|[[818044, 2.54349...|
|144685|[[205937, 3.18195...|
|160767|[[519704, 4.25704...|
|167881|[[1080241, 2.6779...|
|172319|[[204866, 4.89816...|
|173508|[[791739, 2.87190...|
|191355|[[399121, 7.07733...|
+------+--------------------+
only showing top 20 rows



In [0]:
# Saving the result to csv file
userRecs.toPandas().to_csv('Top10ForUserwithImplicit.csv')